## APPLIED DATA SCIENCE CAPSTONE PROJECT - WHERE TO OPEN A COFFEE SHOP IN IOWA CITY, IA



### INTRODUCTION

This project will look at which neighborhood in Iowa City, IA would be best for opening a coffee shop. Iowa City is a city in southeast Iowa that houses the University of Iowa. The city has a population of 76,290, and the University has 32,948 students. The large university offers amble opportunity for coffee shops as they are a popular place for students to study. The coffee shop needs to be located in a neighborhood that will attract students, but also other members of the community in order to remain profitable throughout the year, even when school is not in session. There is a coffee shop chain from St Paul, Minnesota that is looking to open a new branch in Iowa City, and would like to know where would be the best area. 
Where should we suggest that this coffee shop be opened?


### Data

The data used to determine existing venues in the area will come from Foursquare (https://developer.foursquare.com/), as well as data from the city's neighborhood's site (https://www.iowa-city.org/weblink/0/doc/1481280/Electronic.aspx), data from the city's crime map (http://communitycrimemap.com/?address=Iowa-City%20), and data from the 2010 census (https://www.census.gov/quickfacts/iowacitycityiowa)

This data will allow us to look at the foursquare venue data by neighborhood and compare this to crime data, as well as population density data to best predict where a new coffee shop chain in Iowa City will thrive.

